In [62]:
import os
import csv
import re
from collections import defaultdict, Counter
import nltk
from nltk import bigrams, trigrams
import random

In [63]:
def remove_punctuation(word):
    return re.sub(r'[^\w\s]', '', word)

Lettura del trump twitter archive

In [64]:
file_name = 'tweets.csv'
current_dir = os.getcwd()
file_path = os.path.join(current_dir, 'trump_twitter_archive/', file_name)

with open(file_path, 'r', encoding='utf-8') as train:
   prime_10_righe = train.readlines()[1:]

Pulizia dei tweet

In [65]:
tweets = []
for riga in prime_10_righe:
   riga = riga.split(',')
   riga[1] = re.sub(r"http\S+", "", riga[1])
   tweets.append(riga[1])
print(tweets)

['LOSER! ', 'Most of the money raised by the RINO losers of the so-called “Lincoln Project” goes into their own pockets. With what I’ve done on Judges Taxes Regulations Healthcare the Military Vets (Choice!) &amp; protecting our great 2A they should love Trump. Problem is I BEAT THEM ALL!', '....because they don’t know how to win and their so-called Lincoln Project is a disgrace to Honest Abe. I don’t know what Kellyanne did to her deranged loser of a husband Moonface but it must have been really bad. John Weaver lost big for Kasich (to me). Crazed Rick Wilson....', '....lost for Evan “McMuffin” McMullin (to me). Steve Schmidt &amp; Reed Galvin lost for John McCain Romney’s campaign manager (?) lost big to “O” &amp; Jennifer Horn got thrown out of the New Hampshire Republican Party. They’re all LOSERS but Abe Lincoln Republican is all smiles!', '....get even for all of their many failures. You see these loser types don’t care about 252 new Federal Judges 2 great Supreme Court Justices 

Creazione dei brigrammi e trigrammi

In [66]:
# Costruzione delle strutture a bigrammi e trigrammi
model_bigrams = defaultdict(lambda: defaultdict(lambda: 0))
model_trigrams = defaultdict(lambda: defaultdict(lambda: 0))
tot_length = 0

for tweet in tweets:
    words = tweet.split()
    clean_words = [remove_punctuation(word) for word in words]
        
    # Bigrammi
    for w1, w2 in bigrams(clean_words):
        model_bigrams[w1.lower()][w2.lower()] += 1
    
    # Trigrammi
    for w1, w2, w3 in trigrams(clean_words):
        model_trigrams[(w1.lower(), w2.lower())][w3.lower()] += 1
        
print(model_bigrams)

defaultdict(<function <lambda> at 0x000001F132722550>, {'most': defaultdict(<function <lambda>.<locals>.<lambda> at 0x000001F1325D5B80>, {'of': 1, 'others': 1, 'disloyal': 1, 'important': 1, 'people': 2, 'corrupt': 1, 'slanted': 1, 'thought': 1, 'overrated': 1, 'envied': 1}), 'of': defaultdict(<function <lambda>.<locals>.<lambda> at 0x000001F1325D5C10>, {'the': 30, 'a': 4, 'their': 2, 'them': 1, 'bernie': 1, 'massachusetts': 1, 'his': 4, 'south': 1, 'jeb': 1, 'moderate': 1, 'zero': 1, 'relentless': 1, 'michigan': 1, 'everybody': 1, 'control': 1, 'london': 1, 'vicious': 1, 'your': 1, 'campaign': 1, '13': 1, 'trouble': 2, 'big': 1, 'immigration': 1, 'these': 2, 'all': 1, 'scotland': 1, 'defeat': 1, 'philadelphias': 1, 'politics': 2, 'which': 1, 'success': 1, 'haters': 5, 'goldberg': 1, 'losers': 2, 'fate': 6, 'everything': 1, 'ny': 1, 'july': 1, 'this': 1, 'lives': 1, 's': 1, 'yourself': 1, 'blackdog': 1, 'myself': 1, 'time': 2, 'you': 2, 'celebrity': 1, 'successand': 1, '': 1, 'losera':

Calcolo delle probabilità per i bigrammi e i trigrammi

In [67]:
#calcolo probabilità bigrammi
for w1 in model_bigrams:
  tot = sum(model_bigrams[w1].values())
  for w2 in model_bigrams[w1]:
      model_bigrams[w1][w2] /= tot
print(model_bigrams)

defaultdict(<function <lambda> at 0x000001F132722550>, {'most': defaultdict(<function <lambda>.<locals>.<lambda> at 0x000001F1325D5B80>, {'of': 0.09090909090909091, 'others': 0.09090909090909091, 'disloyal': 0.09090909090909091, 'important': 0.09090909090909091, 'people': 0.18181818181818182, 'corrupt': 0.09090909090909091, 'slanted': 0.09090909090909091, 'thought': 0.09090909090909091, 'overrated': 0.09090909090909091, 'envied': 0.09090909090909091}), 'of': defaultdict(<function <lambda>.<locals>.<lambda> at 0x000001F1325D5C10>, {'the': 0.2830188679245283, 'a': 0.03773584905660377, 'their': 0.018867924528301886, 'them': 0.009433962264150943, 'bernie': 0.009433962264150943, 'massachusetts': 0.009433962264150943, 'his': 0.03773584905660377, 'south': 0.009433962264150943, 'jeb': 0.009433962264150943, 'moderate': 0.009433962264150943, 'zero': 0.009433962264150943, 'relentless': 0.009433962264150943, 'michigan': 0.009433962264150943, 'everybody': 0.009433962264150943, 'control': 0.00943396

In [68]:
#calcolo probabilità trigrammi
for w1w2 in model_trigrams:
    tot = sum(model_trigrams[w1w2].values())
    for w3 in model_trigrams[w1w2]:
        model_trigrams[w1w2][w3] /= tot
print(model_trigrams)

defaultdict(<function <lambda> at 0x000001F1325B4430>, {('most', 'of'): defaultdict(<function <lambda>.<locals>.<lambda> at 0x000001F1327E2430>, {'the': 1.0}), ('of', 'the'): defaultdict(<function <lambda>.<locals>.<lambda> at 0x000001F1327E24C0>, {'money': 0.03333333333333333, 'socalled': 0.03333333333333333, 'new': 0.03333333333333333, 'night': 0.03333333333333333, 'dumbest': 0.03333333333333333, 'united': 0.06666666666666667, 'people': 0.03333333333333333, 'president': 0.03333333333333333, 'losers': 0.06666666666666667, 'witch': 0.03333333333333333, 'great': 0.06666666666666667, 'fbi': 0.06666666666666667, 'mainstream': 0.03333333333333333, 'political': 0.03333333333333333, 'conservative': 0.03333333333333333, 'haters': 0.16666666666666666, 'world': 0.06666666666666667, 'repetitive': 0.03333333333333333, 'highest': 0.03333333333333333, 'obama': 0.03333333333333333, 'year': 0.03333333333333333}), ('the', 'money'): defaultdict(<function <lambda>.<locals>.<lambda> at 0x000001F130511C10

GENERAZIONE DI TWEET BASATI SU BIGRAMMI

In [69]:
#questo blocco sceglie una parola randomica tra quelle presenti nei tweet,
#dopodichè produce una frase di 13 parole basandosi sulle probabilità ottenute con i bigrammi

max_lenght = 12

for frase in range(5):   
    first_words = list(model_bigrams.keys())
    current_word = random.choice(first_words)
    bi_tweet = [current_word]
    print('Parola iniziale: ')
    print(current_word)
    
    for i in range(max_lenght):
        #aggiungere il caso in cui non ci sia nessun bigramma disponibile per current_word
        new_word = max(model_bigrams[current_word], key=model_bigrams[current_word].get)
        bi_tweet.append(new_word)
        current_word = new_word
    print(bi_tweet)
    

Parola iniziale: 
husband
['husband', 'moonface', 'but', 'the', 'haters', 'and', 'losers', 'and', 'losers', 'and', 'losers', 'and', 'losers']
Parola iniziale: 
steyer
['steyer', 'is', 'a', 'loser', 'who', 'has', 'no', 'idea', 'how', 'a', 'loser', 'who', 'has']
Parola iniziale: 
angry
['angry', 'that', 'i', 'dont', 'know', 'how', 'a', 'loser', 'who', 'has', 'no', 'idea', 'how']
Parola iniziale: 
average
['average', 'talent', 'and', 'losers', 'and', 'losers', 'and', 'losers', 'and', 'losers', 'and', 'losers', 'and']
Parola iniziale: 
mccabes


ValueError: max() arg is an empty sequence

GENERAZIONE DI TWEET BASATI SU TRIGRAMMI

In [ ]:
#questo blocco sceglie una parola randomica tra quelle presenti nei tweet,
#dopodichè produce una frase di 13 parole basandosi sulle probabilità ottenute con i trigrammi

max_lenght = 12

for frase in range(5):   
    first_words = list(model_bigrams.keys())
    current_word = random.choice(first_words)
    tri_tweet = [current_word]
    print('Parola iniziale: ')
    print(current_word)
    
    #alla prima iterazione abbiamo solo un bigramma
    new_word = max(model_bigrams[current_word], key=model_bigrams[current_word].get)
    tri_tweet.append(new_word)
    prev_word = current_word
    current_word = new_word
    
    #nelle iterazioni successive si usano i trigrammi
    for i in range(max_lenght):
        if model_trigrams[prev_word, current_word]:
           new_word = max(model_trigrams[prev_word, current_word], key=model_trigrams[prev_word, current_word].get)
           max_value = model_trigrams[prev_word, current_word][new_word]
           tri_tweet.append(new_word)
           prev_word = current_word
           current_word = new_word
        else 
        
    print(tri_tweet)

SyntaxError: invalid syntax (2257816093.py, line 27)

In [ ]:
#al momento il controllo sulla punteggiatura elimina le @ degli account taggati, quindi ci sono parole incomprensibili(i nomi degli account senza @)
#fare prove modificando lunghezza frase
#salvare anche i segni di punteggiatura come parole a sè stanti
#si possono aggiungere controlli (se parola già usata nella frase, non ripeterla)
#calcolare lunghezza media dei tweet originali